In [1]:
from pyscf import gto, dft, scf, lib, cc, fci
import numpy as np
import scipy as sp

In [2]:
basis_set = 'cc-pVDZ'
# basis_set = 'sto3g'

In [3]:
%matplotlib notebook

H_bond=0.74
R= 0.74 # distance of H2 --- R --- H2

Ha1=(0, 0, -H_bond)
Ha2= (0, 0, 0)

Hb1= (0,0,R)
Hb2= (0,0,R+H_bond)

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

ax.scatter(*Ha1, marker='o', color='green', label='systemA')
ax.scatter(*Ha2, marker='o', color='green', label='systemA')
ax.scatter(*Hb1, marker='o', color='red', label='systemB')
ax.scatter(*Hb2, marker='o', color='red', label='systemB')
plt.legend()
plt.show()


<IPython.core.display.Javascript object>

### Define systems

In [4]:
mol_A = gto.Mole(atom='{} {} {} {}'.format('H',*Ha1) + "; " + '{} {} {} {}'.format('H',*Ha2),
               basis=basis_set,
               charge=0,
               spin=0)

print(mol_A.atom)
print('')
mol_A.build()

H 0 0 -0.74; H 0 0 0



In [5]:
mol_B = gto.Mole(atom='{} {} {} {}'.format('H',*Hb1) + "; " + '{} {} {} {}'.format('H',*Hb2),
               basis=basis_set,
               charge=0,
               spin=0)

print(mol_B.atom)
print('')
mol_B.build()

H 0 0 0.74; H 0 0 1.48



### Get integrals between subsystems

In [6]:
mol_AB = mol_A + mol_B # define full system!

nao_A =  mol_A.nao_nr()# number atomic orbs system A
nao_B =  mol_B.nao_nr()# number atomic orbs system B

#### One electron integrals!

In [7]:
# overlap matrices S
Overlap_S_full_molecule = mol_AB.intor('int1e_ovlp')
overlap_SAB = gto.intor_cross('int1e_ovlp', mol_A, mol_B)

print(np.allclose(Overlap_S_full_molecule[:nao_A,nao_A:], overlap_SAB))


# kinetic terms
kinetic_TAB = gto.intor_cross('int1e_kin', mol_A, mol_B)
kinetic_T_full_molecule = mol_AB.intor('int1e_kin')

print(np.allclose(kinetic_T_full_molecule[:nao_A,nao_A:], kinetic_TAB))


# nuclear terms
nuclear_VAB = gto.intor_cross('int1e_nuc', mol_A, mol_B)
nuclear_V_full_molecule = mol_AB.intor('int1e_nuc')

print(np.allclose(nuclear_V_full_molecule[:nao_A,nao_A:], nuclear_VAB))

True
True
True


#### Two electron integrals!

In [8]:

# shls_slice : 8-element list
#(ish_start, ish_end, jsh_start, jsh_end, ksh_start, 
# ksh_end, lsh_start, lsh_end)

# a,c (ab|cd) on one molecule A and bd on molecule B


eri_aaaa = mol_AB.intor('int2e', shls_slice=(0, mol_A.nbas,
                                       0, mol_A.nbas,
                                       0, mol_A.nbas,
                                       0, mol_A.nbas))

# (xy|wz) : x,y, on molecule A and w,z on molecule B 
eri_aaab = mol_AB.intor('int2e', shls_slice=(0, mol_A.nbas,
                                        0, mol_A.nbas,
                                        0, mol_A.nbas,
                                        mol_A.nbas, mol_A.nbas+mol_B.nbas))

# (xy|wz) : x,y, on molecule B and w,z on molecule A 
eri_aabb = mol_AB.intor('int2e', shls_slice=(0, mol_A.nbas,
                                        0, mol_A.nbas,
                                        mol_A.nbas, mol_A.nbas+mol_B.nbas,
                                         mol_A.nbas, mol_A.nbas+mol_B.nbas))

# (xy|wz) : x,w, on molecule B and y,z on molecule A
eri_abab = mol_AB.intor('int2e', shls_slice=(0, mol_A.nbas,
                                        mol_A.nbas, mol_A.nbas+mol_B.nbas,
                                       0, mol_A.nbas,
                                         mol_A.nbas, mol_A.nbas+mol_B.nbas))


# (xy|wz) : x,y, on molecule B and w,z on molecule A 
eri_abbb = mol_AB.intor('int2e', shls_slice=(0, mol_A.nbas,
                                        mol_A.nbas, mol_A.nbas+mol_B.nbas,
                                        mol_A.nbas, mol_A.nbas+mol_B.nbas,
                                         mol_A.nbas, mol_A.nbas+mol_B.nbas))

# (xy|wz) : x,y, w, z  on molecule B 
eri_bbbb = mol_AB.intor('int2e', shls_slice=(mol_A.nbas, mol_A.nbas+mol_B.nbas,
                                       mol_A.nbas, mol_A.nbas+mol_B.nbas,
                                       mol_A.nbas, mol_A.nbas+mol_B.nbas,
                                       mol_A.nbas, mol_A.nbas+mol_B.nbas))

testAA = mol_A.intor('int2e')
print(np.allclose(eri_aaaa, testAA))

testBB = mol_B.intor('int2e')
print(np.allclose(eri_bbbb, testBB))


True
True


In [9]:
# applying all symmetry!

eri_total = np.zeros((mol_AB.nao_nr(), 
                      mol_AB.nao_nr(), 
                      mol_AB.nao_nr(), 
                      mol_AB.nao_nr()))

nao_A =  mol_A.nao_nr()# number atomic orbs system A
nao_B =  mol_B.nao_nr()# number atomic orbs system B

# sys A
eri_total[:nao_A,:nao_A,:nao_A,:nao_A]= eri_aaaa

# single B
eri_total[:nao_A,:nao_A,:nao_A,nao_A:]= eri_aaab #ijkl=aaab
eri_total[:nao_A,:nao_A,nao_A:,:nao_A]= np.einsum('ijkl->ijlk', eri_aaab)#ijlk=aaba
eri_total[:nao_A,nao_A:,:nao_A,:nao_A]= np.einsum('ijkl->klij', eri_aaab)#iljk=abaa
eri_total[nao_A:, :nao_A,:nao_A,:nao_A]= np.einsum('ijkl->lkji', eri_aaab)#lijk=baaa

# double B sym
eri_total[:nao_A,:nao_A,nao_A:,nao_A:]= eri_aabb#ijkl=aabb
eri_total[nao_A:,nao_A:,:nao_A,:nao_A]= np.einsum('ijkl->klij', eri_aabb)#iljk=bbaa

# double B anti-sym
eri_total[:nao_A,nao_A:,:nao_A,nao_A:]= eri_abab#ijkl=abab
eri_total[:nao_A,nao_A:,nao_A:,:nao_A]= np.einsum('ijkl->ijlk', eri_abab)#ilkj=abba
eri_total[nao_A:, :nao_A,:nao_A,nao_A:]= np.einsum('ijkl->jikl', eri_abab)#lijk=baab
eri_total[nao_A:, :nao_A,nao_A:,:nao_A]= np.einsum('ijkl->jilk', eri_abab)#lijk=baba


# triple B
eri_total[:nao_A,nao_A:,nao_A:,nao_A:]= eri_abbb #ijkl=abbb
eri_total[nao_A:,:nao_A,nao_A:,nao_A:]= np.einsum('ijkl->jikl', eri_abbb)#ijlk=babb
eri_total[nao_A:,nao_A:,:nao_A,nao_A:]= np.einsum('ijkl->klij', eri_abbb)#iljk=bbab
eri_total[nao_A:,nao_A:,nao_A:,:nao_A]= np.einsum('ijkl->klji', eri_abbb)#lijk=bbba

# sys B
eri_total[nao_A:,nao_A:,nao_A:,nao_A:]= eri_bbbb

TEST = mol_AB.intor('int2e')
np.allclose(eri_total, TEST)

True

In [10]:
# sys B
eri_total[nao_A:,nao_A:,nao_A:,nao_A:]= eri_bbbb

TEST = mol_AB.intor('int2e')
np.allclose(eri_total, TEST)

True

## Notes on method
The total system is split into subsystem A and subsystem B

$$\gamma^{\text{Total}} = \gamma^{A} + \gamma^{B}$$


The Fock matrix of subsystem A embeddeed in subsystem B is [J.Chem. Theory Comput. 2018, **14**, 1928-1942](https://pubs.acs.org/doi/abs/10.1021/acs.jctc.7b01154):

$$F^{A-in-B}[\gamma^{A}_{emb}; \gamma^{A},\gamma^{B}] = h^{A-in-B}[\gamma^{A},\gamma^{B}] + g[\gamma^{A}_{emb}]$$

- $\gamma^{A}_{emb}$ is defined selfconsistently by $F^{A-in-B}$
- note that $ \gamma^{A}$ and $\gamma^{B}$ remain FIXED!

where also:
$$h^{A-in-B}[\gamma^{A},\gamma^{B}] = h^{core}[\gamma^{A}+\gamma^{B}] + g[\gamma^{A}+\gamma^{B}] -g[\gamma^{A}] + P^{B}[\gamma^{B}] $$

Note the Huzinga operator is defined as:

$$P^{B} = -\frac{1}{2} \big(F^{AB} \gamma^{B} S^{BA} +  S^{AB}\gamma^{B} F^{BA} \big)$$


Finally, the overall energy is given by:

$$E[\gamma^{A}_{emb}; \gamma^{A},\gamma^{B}] = \mathcal{E}_{\text{KS-DFT}}[\gamma^{A}_{emb}] +E_{\text{KS-DFT}}[\gamma^{B}] + g[\gamma^{A},\gamma^{B}] + tr\Bigg( \big(\gamma^{A}_{emb}-\gamma^{A} \big) \big(h^{A-in-B}[\gamma^{A},\gamma^{B}] - h^{core}[\gamma^{A}+\gamma^{B}] \big) \Bigg)$$

where:

$$ g[\gamma^{A},\gamma^{B}] = J^{\text{non-additive}}[\gamma^{A},\gamma^{B}] + E_{xc}^{\text{non-additive}}[\gamma^{A},\gamma^{B}]$$

- $ J^{\text{non-additive}}[\gamma^{A},\gamma^{B}] = \int dr_{1} \int dr_{2} \frac{\gamma^{A}(1)\gamma^{B}(2)}{r_{12}} $
- $ E_{xc}^{\text{non-additive}}[\gamma^{A},\gamma^{B}] = E_{xc}[\gamma^{A}+\gamma^{B}] - E_{xc}[\gamma^{A}]-E_{xc}[\gamma^{B}]$


All this is defined in:
- [J.Chem.Phys **139**, 024103 (2013)](https://aip.scitation.org/doi/10.1063/1.4811112)
    - this paper defines $ g[\gamma^{A},\gamma^{B}]$ in equations (6-8)
- [J.Chem. Theory Comput. 2019, **15**, 1053-1064](https://pubs.acs.org/doi/abs/10.1021/acs.jctc.8b01112)

As stated in [J.Chem. Theory Comput. 2019, **15**, 1053-1064](https://pubs.acs.org/doi/abs/10.1021/acs.jctc.8b01112) the wavefunction approach is simply:

$$E[\Psi^{A}; \gamma^{A},\gamma^{B}] = \langle \Psi^{A} |H^{A-in-B}| \Psi^{A} \rangle +E_{\text{KS-DFT}}[\gamma^{B}] + g[\gamma^{A},\gamma^{B}] + tr\Bigg( \gamma^{A} \big(h^{A-in-B}[\gamma^{A},\gamma^{B}] - h^{core}[\gamma^{A}+\gamma^{B}] \big) \Bigg)$$

Note that [J.Chem. Theory Comput. 2020, **16**, 2284-2295](https://pubs.acs.org/doi/10.1021/acs.jctc.9b01185) $H^{A-in-B}$ is defined in equation 12 as:

$$H^{A-in-B} = h^{A-in-B}[\gamma^{A},\gamma^{B}]  + g^{A}$$

- where $ g^{A}$ is the two-electron operator for the given WF theory action on the electrons in subsystem A

**may be important to check paragraph below equation 9 in:** [J.Chem.Phys **139**, 024103 (2013)](https://aip.scitation.org/doi/10.1063/1.4811112)

In [11]:
mol_A_KSDFT = scf.RKS(mol_A)
mol_A_KSDFT.xc = 'lda,vwn'
mol_A_KSDFT.small_rho_cutoff = 1e-20
mol_A_KSDFT.kernel()
gamma_A = mol_A_KSDFT.make_rdm1()

mol_T_KSDFT = scf.RKS(mol_AB)
mol_T_KSDFT.xc = 'lda,vwn'
mol_T_KSDFT.small_rho_cutoff = 1e-20
mol_T_KSDFT.kernel()
gamma_T = mol_T_KSDFT.make_rdm1()

print(np.allclose(gamma_T[:nao_A,:nao_A], gamma_A))
# clearly NOT the same!

converged SCF energy = -1.13141128387478
converged SCF energy = -2.17958287616601
False


### The algorithm!

Follow bottom left page (1929) of [J.Chem. Theory Comput. 2018, **14**, 1928-1942](https://pubs.acs.org/doi/abs/10.1021/acs.jctc.7b01154) 

1. We can construct both an embedded core Hamiltonian for A-in-B and B-in-A!
    - where B-in-A will need the proejction operator $P^{A}$ from subsystem A
2. Goal is to solve these embedded Fock matrices $F^{A-in-B}$ and $F^{B-in-A}$ self-consistently
    - this is termed DFT-in-DFT
    - this gives the **ground state densities**: $\gamma^{A}_{0}$ and $\gamma^{B}_{0}$
    - in detail  [J.Chem. Theory Comput. 2017, **13**, 1503-1508](https://pubs.acs.org/doi/abs/10.1021/acs.jctc.7b00034) :
        - perform iterative DFT-in-DFT calculations in the mono-molecular basis
        - **alternating between sybsystems, keeping the other frozen**
        - upon convergence perform a monomolucular WF-in-DFt calcultion
3. Finally perform a WF-in-DFT calculation

$$h^{A-in-B}[\gamma^{A},\gamma^{B}] = h^{core}[\gamma^{A}+\gamma^{B}] + g[\gamma^{A}+\gamma^{B}] -g[\gamma^{A}] + P^{B}[\gamma^{B}] $$

In [12]:
def embedded_core_H_XinY(h_core_FULL, g_full, g_subX, P_subY):
    
    H_XinY = h_core_FULL + g_full - g_subX + P_subY
    
    return H_XinY


$$P^{B} = -\frac{1}{2} \big(F^{AB} \gamma^{B} S^{BA} +  S^{AB}\gamma^{B} F^{BA} \big)$$

In [13]:
def HuzingaOp_Y(P_Y, Fock_XY, gammaY, Overlap_S_YX):
    F_XY_gammaY_S_YX = np.dot(Fock_XY, gammaY.dot(Overlap_S_YX))
    S_XY_gammaY_F_YX = F_XY_gammaY_S_YX.T #transpose
    
    P_y = -0.5*(F_XY_gammaY_S_YX + S_XY_gammaY_F_YX)
    
    return P_y

$$F^{A-in-B}[\gamma^{A}_{emb}; \gamma^{A},\gamma^{B}] = h^{A-in-B}[\gamma^{A},\gamma^{B}] + g[\gamma^{A}_{emb}]$$

In [14]:
# def Fock_XinY(Hcore_XinY, gammaX, gammaY, gammaX_embedded):
#     F_XinY = 
#     return F_XinY

Need to solve iteratively (alternativing between subsystems):

$$F^{A-in-B}[\gamma^{A}_{emb}; \gamma^{A},\gamma^{B}] = h^{A-in-B}[\gamma^{A},\gamma^{B}] + g[\gamma^{A}_{emb}]$$

Note that $h^{A-in-B}[\gamma^{A},\gamma^{B}]$ **remains CONSTANT throughout!!!**


$$h^{A-in-B}[\gamma^{A},\gamma^{B}] = h^{core}[\gamma^{A}+\gamma^{B}] + g[\gamma^{A}+\gamma^{B}] -g[\gamma^{A}] + P^{B}[\gamma^{B}] $$

Note the Huzinga operator is defined as:

$$P^{B} = -\frac{1}{2} \big(F^{AB} \gamma^{B} S^{BA} +  S^{AB}\gamma^{B} F^{BA} \big)$$


In [16]:
from scipy import linalg as LA
from pyscf import gto, scf


In [17]:
# Fock_DFT = one_electron + nuclear_electron + couloumb + exchange

F_AB = gto.intor_cross('int1e_kin', mol_A, mol_B) \
       + gto.intor_cross('int1e_nuc', mol_A, mol_B) \
       + mol_T_KSDFT.get_hcore()[:nao_A,nao_A:] \
       + mol_T_KSDFT.get_veff()[:nao_A,nao_A:]

overlap_S_BA = gto.intor_cross('int1e_ovlp', mol_B, mol_A)

mol_B_KSDFT = scf.RKS(mol_B)
mol_B_KSDFT.xc = 'lda,vwn'
mol_B_KSDFT.small_rho_cutoff = 1e-20
gammaB= mol_B_KSDFT.init_guess_by_atom()


F_AB_gammaB_S_BA = np.dot(F_AB, gammaB.dot(overlap_S_BA))
S_AB_gammaY_F_BA = F_AB_gammaB_S_BA.T #transpose
P_B = -0.5*(F_AB_gammaB_S_BA + S_AB_gammaY_F_BA)


mol_T_KSDFT = scf.RKS(mol_AB)
mol_T_KSDFT.xc = 'lda,vwn'
mol_T_KSDFT.small_rho_cutoff = 1e-20
H_core_full = mol_T_KSDFT.get_hcore()
gammaA_plus_gammaB = mol_T_KSDFT.init_guess_by_atom()
G_full = mol_T_KSDFT.get_veff(dm=gammaA_plus_gammaB)


mol_A_KSDFT = scf.RKS(mol_A)
mol_A_KSDFT.xc = 'lda,vwn'
mol_A_KSDFT.small_rho_cutoff = 1e-20
gammaA= mol_A_KSDFT.init_guess_by_atom()
G_subA = mol_A_KSDFT.get_veff(dm=gammaA)


Hcore_AinB = H_core_full[:nao_A,nao_A:] + G_full[:nao_A,nao_A:] - G_subA + P_B
# Hcore_AinB is fixed!


In [ ]:
# ### customise core Hamiltonian!
# # mol_A_MODIFIED_KSDFT = scf.RKS(mol_A)
# # mol_A_MODIFIED_KSDFT.get_hcore = lambda *args: Hcore_AinB
# # mol_A_MODIFIED_KSDFT.xc = 'lda,vwn'
# # mol_A_MODIFIED_KSDFT.small_rho_cutoff = 1e-20

# # mol_A_MODIFIED_KSDFT.kernel()


# ## orthogonalizing matrix S^-0.5
# # U†SU = L
# eig_vals, eig_vecs = LA.eig(overlap_SAB) ### <--- not sure whether this should be overlap of only S_AA
# U_dag =np.conjugate(eig_vecs).T
# U=eig_vecs
# L=U_dag @ overlap_SAB @ U
# # # Form S^U(-0.5)
# S_half = U @ LA.fractional_matrix_power(L, -0.5) @ U_dag

# gammaA_embedded= mol_A_KSDFT.init_guess_by_atom()
# Fock_AinB = Hcore_AinB + mol_A_KSDFT.get_veff(dm=gammaA_embedded)

# for i in range(50):
#         # transofrm Fock matrix to orthonormal basis
#         Fockprime = np.dot(np.conjugate(S_half).T,np.dot(Fock_AinB, S_half))

#         # diagonalise Fock matrix
#         evalFockprime, Cprime = np.linalg.eig(Fockprime)

#         idx = evalFockprime.argsort()
#         evalFockprime = evalFockprime[idx]
#         Cprime = Cprime[:,idx]
        
#         ## construct new SCF eigenvector matrix
#         C_new = S_half@Cprime 
    
#         ## form new density matrix
#         Dnew=np.zeros((mol_A.nao, mol_A.nao), dtype=complex)
#         N=mol_A.nao
#         for j in range(N):
#             for k in range(N):
#                 for a in range(int(N/2)):
#                     Dnew[j,k] += C_new[j,a]*C_new[k,a]
                    
#         Fock_AinB = Hcore_AinB + mol_A_KSDFT.get_veff(dm=Dnew)
        
# #         if i>0:
# #             error = sp.linalg.norm(Dnew - Dold) / sp.linalg.norm(Dold)
# #             print(error)
# #         Dold = Dnew.copy()
# Dnew

In [21]:
### customise core Hamiltonian!
# mol_A_MODIFIED_KSDFT = scf.RKS(mol_A)
# mol_A_MODIFIED_KSDFT.get_hcore = lambda *args: Hcore_AinB
# mol_A_MODIFIED_KSDFT.xc = 'lda,vwn'
# mol_A_MODIFIED_KSDFT.small_rho_cutoff = 1e-20

# mol_A_MODIFIED_KSDFT.kernel()

overlap_SAA= mol_A.intor('int1e_ovlp') ### <--- overlap of only S_AA
## orthogonalizing matrix S^-0.5
# U†SU = L
eig_vals, eig_vecs = LA.eig(overlap_SAA)
U_dag =np.conjugate(eig_vecs).T
U=eig_vecs
L=U_dag @ overlap_SAA @ U
# # Form S^U(-0.5)
S_half = U @ LA.fractional_matrix_power(L, -0.5) @ U_dag

gammaA_embedded= mol_A_KSDFT.init_guess_by_atom()
Fock_AinB = Hcore_AinB + mol_A_KSDFT.get_veff(dm=gammaA_embedded)

for i in range(50):
        # transofrm Fock matrix to orthonormal basis
        Fockprime = np.dot(np.conjugate(S_half).T,np.dot(Fock_AinB, S_half))

        # diagonalise Fock matrix
        evalFockprime, Cprime = np.linalg.eig(Fockprime)

        idx = evalFockprime.argsort()
        evalFockprime = evalFockprime[idx]
        Cprime = Cprime[:,idx]
        
        ## construct new SCF eigenvector matrix
        C_new = S_half@Cprime 
    
        ## form new density matrix
        Dnew=np.zeros((mol_A.nao, mol_A.nao), dtype=complex)
        N=mol_A.nao
        for j in range(N):
            for k in range(N):
                for a in range(int(N/2)):
                    Dnew[j,k] += C_new[j,a]*C_new[k,a]
                    
        Fock_AinB = Hcore_AinB + mol_A_KSDFT.get_veff(dm=Dnew)
        
#         E=mol_A_MODIFIED_KSDFT.energy_elec(dm=Dnew)
#         if i>0:
#             error = sp.linalg.norm(Dnew - Dold) / sp.linalg.norm(Dold)
# #             print(error)
#             print(E[0]-Eold)
#         Eold = E[0].copy()
#         Dold = Dnew.copy()
# Dnew

In [30]:
## customise core Hamiltonian!
mol_A_MODIFIED_KSDFT = scf.RKS(mol_A)
# mol_A_MODIFIED_KSDFT = scf.density_fit(scf.RKS(mol_A))

mol_A_MODIFIED_KSDFT.get_hcore = lambda *args: Hcore_AinB
mol_A_MODIFIED_KSDFT.xc = 'lda,vwn'

mol_A_MODIFIED_KSDFT.kernel()
mol_A_MODIFIED_KSDFT.make_rdm1()

SCF not converged.
SCF energy = -0.376774058199502


array([[ 8.26467233e+00,  2.18106276e+00,  1.02364243e-03,
        -2.34045214e-03,  8.16485553e+00, -7.36597870e+00,
        -2.70633358e+00,  9.62148466e-04,  2.50103296e-03,
         7.34464837e+00],
       [ 2.18106276e+00,  5.75586616e-01,  2.70141187e-04,
        -6.17649776e-04,  2.15472092e+00, -1.94389580e+00,
        -7.14206585e-01,  2.53912812e-04,  6.60027360e-04,
         1.93826669e+00],
       [ 1.02364243e-03,  2.70141187e-04,  1.26785889e-07,
        -2.89882772e-07,  1.01127936e-03, -9.12332403e-04,
        -3.35199967e-04,  1.19169394e-07,  3.09771926e-07,
         9.09690479e-04],
       [-2.34045214e-03, -6.17649776e-04, -2.89882772e-07,
         6.62786860e-07, -2.31218527e-03,  2.08595331e-03,
         7.66399921e-04, -2.72468447e-07, -7.08261345e-07,
        -2.07991283e-03],
       [ 8.16485553e+00,  2.15472092e+00,  1.01127936e-03,
        -2.31218527e-03,  8.06624426e+00, -7.27701590e+00,
        -2.67364776e+00,  9.50528092e-04,  2.47082667e-03,
         7.

In [36]:
### customise core Hamiltonian!
# mol_A_MODIFIED_KSDFT = scf.RKS(mol_A)
# mol_A_MODIFIED_KSDFT.get_hcore = lambda *args: Hcore_AinB
# mol_A_MODIFIED_KSDFT.xc = 'lda,vwn'
# mol_A_MODIFIED_KSDFT.small_rho_cutoff = 1e-20

# mol_A_MODIFIED_KSDFT.kernel()

overlap_SAA= mol_A.intor('int1e_ovlp') ### <--- overlap of only S_AA

gammaA_embedded= mol_A_KSDFT.init_guess_by_atom()
Fock_AinB = Hcore_AinB + mol_A_KSDFT.get_veff(dm=gammaA_embedded)
for i in range(50):

        # diagonalise Fock matrix
        mo_energy, mo_coeff = mol_A_MODIFIED_KSDFT.eig(Fock_AinB, overlap_SAA)
        mo_occ = mol_A_MODIFIED_KSDFT.get_occ(mo_energy, mo_coeff)
        Dnew = mol_A_MODIFIED_KSDFT.make_rdm1(mo_coeff, mo_occ)
        
        Fock_AinB = Hcore_AinB + mol_A_MODIFIED_KSDFT.get_veff(dm=Dnew)
        
        e_tot = mol_A_MODIFIED_KSDFT.energy_tot(Dnew, Hcore_AinB, mol_A_KSDFT.get_veff(dm=Dnew))
        
        if i>0:
            error = sp.linalg.norm(Dnew - Dold) / sp.linalg.norm(Dold)
#             print(error)
#             print(e_tot)
        Dold = Dnew.copy()
Dnew

array([[ 8.03329923e+00,  2.06687206e+00,  1.47614227e-15,
        -9.01488783e-16,  8.21927029e+00, -7.50885605e+00,
        -2.43619472e+00, -2.87178708e-16,  1.36676067e-16,
         7.03264298e+00],
       [ 2.06687206e+00,  5.31781522e-01,  3.79793797e-16,
        -2.31942309e-16,  2.11472019e+00, -1.93193908e+00,
        -6.26803837e-01, -7.38876556e-17,  3.51651216e-17,
         1.80941514e+00],
       [ 1.47614227e-15,  3.79793797e-16,  2.71245468e-31,
        -1.65651206e-31,  1.51031500e-15, -1.37977430e-15,
        -4.47657918e-16, -5.27699291e-32,  2.51146279e-32,
         1.29226876e-15],
       [-9.01488783e-16, -2.31942309e-16, -1.65651206e-31,
         1.01164167e-31, -9.22358269e-16,  8.42636296e-16,
         2.73387328e-16,  3.22269066e-32, -1.53376512e-32,
        -7.89196143e-16],
       [ 8.21927029e+00,  2.11472019e+00,  1.51031500e-15,
        -9.22358269e-16,  8.40954659e+00, -7.68268624e+00,
        -2.49259268e+00, -2.93826902e-16,  1.39840122e-16,
         7.

In [67]:
def DFT_in_DFT(full_syst_SCF, active_SCF, environ_SCF, mol_obj_active, mol_obj_environment, max_iter=100,
              gamma_Active=None, gamma_Env=None):
    
    nao_active =  mol_obj_active.nao_nr()# number atomic orbs in active system
    nao_environ =  mol_obj_environment.nao_nr()# number atomic orbs in environment
    

    # Fock_DFT = one_electron + nuclear_electron + couloumb + exchange
    F_Active_Envirnoment = gto.intor_cross('int1e_kin', mol_obj_active, mol_obj_environment) \
           + gto.intor_cross('int1e_nuc', mol_obj_active, mol_obj_environment) \
           + mol_T_KSDFT.get_hcore()[:nao_active,nao_active:] \
           + mol_T_KSDFT.get_veff()[:nao_active,nao_active:]

    overlap_S_ENV_Active = gto.intor_cross('int1e_ovlp', mol_obj_environment, mol_obj_active)
    
    if gamma_Env is None:
        gamma_Env= environ_SCF.init_guess_by_atom()


    F_AE_gammaE_S_EA = np.dot(F_Active_Envirnoment, gamma_Env.dot(overlap_S_ENV_Active))
    S_AE_gammaE_F_EA = F_AE_gammaE_S_EA.T #transpose
    P_E = -0.5*(F_AE_gammaE_S_EA + S_AE_gammaE_F_EA)

    H_core_full = full_syst_SCF.get_hcore() # full system Hcore (standard!)
    gammaA_plus_gammaE = full_syst_SCF.init_guess_by_atom() # full system!
    G_full = full_syst_SCF.get_veff(dm=gammaA_plus_gammaE)

    if gamma_Active is None:
        gamma_Active= active_SCF.init_guess_by_atom()
        
    G_subA = active_SCF.get_veff(dm=gamma_Active)

    
    # this is the modified core Hamiltonian (that stays constant!)
    Hcore_AinE = H_core_full[:nao_active,nao_active:] + G_full[:nao_active,nao_active:] - G_subA + P_E
    
    ### customise core Hamiltonian!
#     mol_A_MODIFIED_KSDFT = active_SCF.copy()
    mol_A_MODIFIED_KSDFT = scf.RKS(mol_obj_active)
    mol_A_MODIFIED_KSDFT.xc = active_SCF.xc
    
    mol_A_MODIFIED_KSDFT.get_hcore = lambda *args: Hcore_AinE # change core Hamiltonian to modified!


    ## Run self-consistent method!
    overlap_SAA= mol_obj_active.intor('int1e_ovlp') ### <--- overlap of only active part only
    
    gammaA_embedded= mol_A_MODIFIED_KSDFT.init_guess_by_atom() # guess of active embedded denisty
    Fock_AinB = Hcore_AinB + mol_A_MODIFIED_KSDFT.get_veff(dm=gammaA_embedded)
    
    for i in range(max_iter):

        # diagonalise Fock matrix
        mo_energy, mo_coeff = mol_A_MODIFIED_KSDFT.eig(Fock_AinB, overlap_SAA)
        mo_occ = mol_A_MODIFIED_KSDFT.get_occ(mo_energy, mo_coeff)
        gammaA_embedded_NEW = mol_A_MODIFIED_KSDFT.make_rdm1(mo_coeff, mo_occ)

        Fock_AinB = Hcore_AinB + mol_A_MODIFIED_KSDFT.get_veff(dm=gammaA_embedded_NEW)

        e_tot = mol_A_MODIFIED_KSDFT.energy_tot(gammaA_embedded_NEW,
                                                Hcore_AinB,
                                                mol_A_KSDFT.get_veff(dm=gammaA_embedded_NEW))
        if i>0:
            error = sp.linalg.norm(gammaA_embedded_NEW - gammaA_embedded_OLD) / sp.linalg.norm(gammaA_embedded_OLD)
#             print(error)
#             print(e_tot)

            if error<1e-5:
                print('converged!')
                print('E_total = ', e_tot)
                break
        gammaA_embedded_OLD = gammaA_embedded_NEW.copy()


    return gammaA_embedded_NEW

'lda,vwn'

In [180]:
mol_A_KSDFT = scf.RKS(mol_A)
mol_A_KSDFT.xc = 'lda,vwn'
mol_A_KSDFT.small_rho_cutoff = 1e-20
mol_A_KSDFT.kernel()
gamma_A = mol_A_KSDFT.make_rdm1()

mol_B_KSDFT = scf.RKS(mol_B)
mol_B_KSDFT.xc = 'lda,vwn'
mol_B_KSDFT.small_rho_cutoff = 1e-20
mol_B_KSDFT.kernel()
gamma_B = mol_B_KSDFT.make_rdm1()


mol_T_KSDFT = scf.RKS(mol_AB)
mol_T_KSDFT.xc = 'lda,vwn'
mol_T_KSDFT.small_rho_cutoff = 1e-20
mol_T_KSDFT.kernel()
gamma_T = mol_T_KSDFT.make_rdm1()



### iterative DFT in DFT calc:
gamma_active = None
gamma_env = None

for j in range(10):
    
    if j%2==0:
        ## subsystem A is active
        
        if gamma_active is None:
            gamma_active = mol_A_KSDFT.make_rdm1()
        else:
            gamma_active = gamma_env.copy() # from previous run
            
        if gamma_env is None:
             gamma_env = mol_B_KSDFT.make_rdm1()
        else:
            gamma_env = gamma_active.copy() # from previous run
            

        gamma_active = DFT_in_DFT(mol_T_KSDFT, mol_A_KSDFT, mol_B_KSDFT, mol_A, mol_B,
                      max_iter=100,
                      gamma_Active=gamma_active,
                       gamma_Env=gamma_env)
        
        gamma_A_optimised = gamma_active.copy()
    
    else:
        ## subsystem B is active

        gamma_active = gamma_env.copy() # from previous run
        gamma_env = gamma_active.copy() # from previous run
  
        gamma_active = DFT_in_DFT(mol_T_KSDFT, mol_B_KSDFT, mol_A_KSDFT, mol_B, mol_A,
                      max_iter=100,
                      gamma_Active=gamma_active,
                       gamma_Env=gamma_env)
    
        gamma_B_optimised = gamma_active.copy()


### Now have gamma_A_optimized and gamma_B_optimized

converged SCF energy = -1.13141128387478
converged SCF energy = -1.13141128387478
converged SCF energy = -2.17958287616601
converged!
E_total =  -0.363179637762437
converged!
E_total =  -0.36317963776247275
converged!
E_total =  -0.36317963776242035
converged!
E_total =  -0.363179637762455
converged!
E_total =  -0.3631796377624572
converged!
E_total =  -0.3631796377624372
converged!
E_total =  -0.363179637762419
converged!
E_total =  -0.36317963776248363
converged!
E_total =  -0.36317963776243345
converged!
E_total =  -0.3631796377624459
converged!
E_total =  -0.3631796377624328
converged!
E_total =  -0.36317963776241013
converged!
E_total =  -0.3631796377624217
converged!
E_total =  -0.3631796377624599
converged!
E_total =  -0.36317963776245565
converged!
E_total =  -0.3631796377624328
converged!
E_total =  -0.36317963776247986
converged!
E_total =  -0.36317963776244055
converged!
E_total =  -0.363179637762447
converged!
E_total =  -0.36317963776246787


Finally, the overall energy is given by:

$$E[\gamma^{A}_{emb}; \gamma^{A},\gamma^{B}] = \mathcal{E}_{\text{KS-DFT}}[\gamma^{A}_{emb}] +E_{\text{KS-DFT}}[\gamma^{B}] + g[\gamma^{A},\gamma^{B}] + tr\Bigg( \big(\gamma^{A}_{emb}-\gamma^{A} \big) \big(h^{A-in-B}[\gamma^{A},\gamma^{B}] - h^{core}[\gamma^{A}+\gamma^{B}] \big) \Bigg)$$

where:

$$ g[\gamma^{A},\gamma^{B}] = J^{\text{non-additive}}[\gamma^{A},\gamma^{B}] + E_{xc}^{\text{non-additive}}[\gamma^{A},\gamma^{B}]$$

- $ J^{\text{non-additive}}[\gamma^{A},\gamma^{B}] = \int dr_{1} \int dr_{2} \frac{\gamma^{A}(1)\gamma^{B}(2)}{r_{12}} $
- $ E_{xc}^{\text{non-additive}}[\gamma^{A},\gamma^{B}] = E_{xc}[\gamma^{A}+\gamma^{B}] - E_{xc}[\gamma^{A}]-E_{xc}[\gamma^{B}]$

In [188]:
mol_A_KSDFT = scf.RKS(mol_A)
mol_A_KSDFT.xc = 'lda,vwn'
mol_A_KSDFT.kernel()
gamma_A_NORMAL = mol_A_KSDFT.make_rdm1()

mol_A_KSDFT_BETTER_FUNCTIONAL = scf.RKS(mol_A)
mol_A_KSDFT_BETTER_FUNCTIONAL.xc = 'b3lyp'
mol_A_KSDFT_BETTER_FUNCTIONAL.get_hcore = lambda *args: Hcore_AinB # Not sure whether tochange core Hamiltonian to modified!

mol_B_KSDFT = scf.RKS(mol_B)
mol_B_KSDFT.xc = 'lda,vwn'
mol_B_KSDFT.kernel()
gamma_B_NORMAL = mol_B_KSDFT.make_rdm1()

mol_T_KSDFT = scf.RKS(mol_AB)
mol_T_KSDFT.kernel()
gamma_T = mol_T_KSDFT.make_rdm1()

## TODO check if this trace operation is correction!
# e_coul = np.einsum('ij,ji->', mol_T_KSDFT.get_k(dm=gamma_T), gamma_T) * .5
# https://sunqm.github.io/pyscf/gto.html ... search numpy.einsum('ij,ji->', vhf, dm) * .5
E_xc_non_add = 0.5*np.trace(mol_T_KSDFT.get_k(dm=gamma_T).dot(gamma_T)) \
- 0.5*np.trace(mol_A_KSDFT.get_k(dm=gamma_A_NORMAL).dot(gamma_A_NORMAL)) \
- 0.5*np.trace(mol_B_KSDFT.get_k(dm=gamma_B_NORMAL).dot(gamma_B_NORMAL)) 

J_non_add = np.trace(mol_T_KSDFT.get_j(dm=gamma_T).dot(gamma_T)) \
-  np.trace(mol_A_KSDFT.get_j(dm=gamma_A_NORMAL).dot(gamma_A_NORMAL)) \
-  np.trace(mol_B_KSDFT.get_j(dm=gamma_B_NORMAL).dot(gamma_B_NORMAL)) 

first_order_correction = np.trace((gamma_A_optimised-gamma_A_NORMAL).dot(Hcore_AinB - mol_T_KSDFT.get_hcore()[:nao_A,nao_A:]))  

E_full = mol_A_KSDFT_BETTER_FUNCTIONAL.energy_tot(gamma_A_optimised,
                                                mol_A_KSDFT_BETTER_FUNCTIONAL.get_hcore(), 
                                                mol_A_KSDFT_BETTER_FUNCTIONAL.get_veff(dm=gamma_A_optimised)) \
        + mol_B_KSDFT.energy_tot() \
        + J_non_add \
        + E_xc_non_add + first_order_correction
        
        
E_full

converged SCF energy = -1.13141128366178
converged SCF energy = -1.13141128366178
converged SCF energy = -2.17958287608859


-2.2473087026549567

In [183]:
HF_full = scf.RHF(mol_AB)
HF_full.kernel()
print('^^^ HFock energy first \n')

CC_scf = cc.CCSD(HF_full)
ecc, t1, t2 = CC_scf.kernel()
CC_scf.e_tot

converged SCF energy = -2.14352527063723
^^^ HFock energy first 

E(CCSD) = -2.221975133322818  E_corr = -0.0784498626855845


-2.221975133322818

# Need to check integrals and traces!

- may be missing a factor of 0.5 in Coulomb term!

In [178]:
0.5*np.trace(mol_T_KSDFT.get_j(dm=gamma_T).dot(gamma_T)) \
-  0.5*np.trace(mol_A_KSDFT.get_j(dm=gamma_A_NORMAL).dot(gamma_A_NORMAL)) \
-  0.5*np.trace(mol_B_KSDFT.get_j(dm=gamma_B_NORMAL).dot(gamma_B_NORMAL)) 


1.4682590985466606

In [170]:
J = np.einsum('ij,ji->', mol_T_KSDFT.get_j(dm=gamma_T)[:nao_A, nao_A:], gamma_T[:nao_A, nao_A:]) * .5
J

-0.6169876847140022

In [133]:
 mol_AB.intor('int2e', aosym='s8').shape

(20, 20, 20, 20)

In [154]:
from pyscf import ao2mo
eri_AB = gto.intor_cross('int2e', mol_A, mol_B)
mo_ints = ao2mo.kernel(mol_AB, mol_T_KSDFT.mo_coeff, eri_AB)

vj, vk = hf.dot_eri_dm(mo_ints, dm=gamma_T)

In [152]:
# from pyscf.scf import hf
# # https://sunqm.github.io/pyscf/_modules/pyscf/scf/hf.html#SCF.get_jk

# eri = mol_AB.intor('int2e', aosym='s8')
# vj_check, vk_check = hf.dot_eri_dm(eri, gamma_T)

# np.allclose(vj_check, vj)

False

In [121]:
J_non_add = np.trace(mol_T_KSDFT.get_j(dm=gamma_T).dot(gamma_T)) \
-  np.trace(mol_A_KSDFT.get_j(dm=gamma_A_NORMAL).dot(gamma_A_NORMAL)) \
- np.trace(mol_B_KSDFT.get_j(dm=gamma_B_NORMAL).dot(gamma_B_NORMAL)) 
J_non_add

2.9365181970933287

In [122]:
np.trace(mol_T_KSDFT.get_j(dm=gamma_T).dot(gamma_T))
np.einsum('ij,ij', gamma_T, mol_T_KSDFT.get_j(dm=gamma_T)).real

8.144406101063534

In [200]:
full_XC_and_J = np.einsum('ij,ji->', mol_T_KSDFT.get_veff(dm=gamma_T), gamma_T) * .5
SubA_XC_and_J = np.einsum('ij,ji->', mol_A_KSDFT.get_veff(dm=gamma_A_NORMAL), gamma_A_NORMAL) * .5
SubB_XC_and_J = np.einsum('ij,ji->', mol_B_KSDFT.get_veff(dm=gamma_B_NORMAL), gamma_B_NORMAL) * .5

Non_add_G = full_XC_and_J-SubA_XC_and_J-SubB_XC_and_J
Non_add_G

1.4121643903024457

In [198]:
E_xc_non_add = 0.5*np.trace(mol_T_KSDFT.get_k(dm=gamma_T).dot(gamma_T)) \
- 0.5*np.trace(mol_A_KSDFT.get_k(dm=gamma_A_NORMAL).dot(gamma_A_NORMAL)) \
- 0.5*np.trace(mol_B_KSDFT.get_k(dm=gamma_B_NORMAL).dot(gamma_B_NORMAL)) 

J_non_add = np.trace(mol_T_KSDFT.get_j(dm=gamma_T).dot(gamma_T)) \
-  np.trace(mol_A_KSDFT.get_j(dm=gamma_A_NORMAL).dot(gamma_A_NORMAL)) \
-  np.trace(mol_B_KSDFT.get_j(dm=gamma_B_NORMAL).dot(gamma_B_NORMAL)) 

E_xc_non_add+J_non_add

3.0400648285920733

In [202]:
E_full = mol_A_KSDFT_BETTER_FUNCTIONAL.energy_tot(gamma_A_optimised,
                                                mol_A_KSDFT_BETTER_FUNCTIONAL.get_hcore(), 
                                                mol_A_KSDFT_BETTER_FUNCTIONAL.get_veff(dm=gamma_A_optimised)) \
        + mol_B_KSDFT.energy_tot() \
        + Non_add_G + 0.5*first_order_correction
        
        
E_full

-1.9823787723672723